  # Apply encoder to morphological profiles to get latent space representations :

In [1]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import sys
import argparse
import csv
import getpass
import json
import time
import types, copy, pprint
import logging 
import re
import shutil
import yaml
from datetime import datetime
for p in ['./src','../..']:
    if p not in sys.path:
        print(f"insert {p}")
        sys.path.insert(0, p)
        
import wandb
import glob
from matplotlib import pyplot as plt
from typing import Dict, List, Tuple
from scipy.sparse import csr_matrix
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from torchinfo import summary

torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=150, profile=None, sci_mode=None)
print(torch.__version__)

pd.options.display.width = 132
pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=150, profile=None, sci_mode=None)

os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Train.ipynb"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
 
timestamp = datetime.now().strftime('%Y_%m_%d_%H:%M:%S')

print(' Timestamp: ','./tmp/sql_cohort'+'_'+timestamp+'.txt')
# logLevel = os.environ.get('LOG_LEVEL', 'INFO').upper()
# FORMAT = '%(asctime)s - %(levelname)s: - %(message)s'
# logging.basicConfig(level="INFO", format= FORMAT)

insert ./src
insert ../..
2.2.1+cu118
 Timestamp:  ./tmp/sql_cohort_2024_06_03_15:20:06.txt


In [3]:
import snnl.utils as utils
# from snnl.utils import parse_args, get_hyperparameters, set_global_seed, get_device, set_device
# from snnl.utils import load_model, save_model, import_results, export_results, save_checkpoint, load_checkpoint 
# from snnl.utils import plot_model_parms, plot_train_history, plot_train_metrics
# from snnl.utils import display_epoch_metrics, display_cellpainting_batch
# from snnl.utils import CellpaintingDataset, InfiniteDataLoader, custom_collate_fn
# from snnl.utils import display_model_summary
# from snnl.utils import load_model_from_ckpt, load_checkpoint_v2, run_model_on_test_data
# from snnl.utils import compute_classification_metrics, compute_regression_metrics
# from snnl.utils import display_classification_metrics, display_regr_metrics, plot_classification_metrics,plot_TSNE, plot_TSNE_2
from KevinsRoutines.utils.utils_general  import  list_namespace
# from KevinsRoutines.utils.utils_wandb  import  init_wandb, wandb_log_metrics,wandb_watch

from dev_base import Model
from dev_dnn import DNN 
from dev_autoencoder import Autoencoder
from dev_snnloss import SNNLoss

In [4]:
import sklearn.metrics as skm 
import scipy.stats as sps 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform, euclidean
import matplotlib.pyplot as plt
import seaborn as sb

# from sklearn.metrics import accuracy_score, f1_score, roc_curve, roc_auc_score, classification_report, precision_recall_fscore_support
# from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, roc_curve, RocCurveDisplay
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [5]:
current_device = utils.get_device()
print(f"Current device is : {current_device}")

current_device = utils.set_device(2)

 device: 0   Quadro GV100                   :  free: 33,170,194,432 B   (30.89 GB)    total: 34,069,872,640 B   (31.73 GB)
 device: 1   Quadro GV100                   :  free: 33,423,949,824 B   (31.13 GB)    total: 34,069,872,640 B   (31.73 GB)
 device: 2   NVIDIA TITAN Xp                :  free: 8,095,662,080 B   (7.54 GB)    total: 12,774,539,264 B   (11.90 GB)
 Current CUDA Device is: cuda:0 Quadro GV100 0
Current device is : cuda:0
 Running on: Quadro GV100 0
 Switch to cuda:2 
 Running on: NVIDIA TITAN Xp 2


## Helper Routines 

# Setup

#### main(args)

In [6]:
# if __name__ == "__main__":
input_args = f" --seed              1234 " \
             f" --runmode           snnl" \
             f" --configuration     hyperparameters/autoencoder_cellpainting-2.yaml"
             
             # f" --runmode              baseline" \
             # f" --configuration     hyperparameters/dnn_mnist.json"
args = utils.parse_args(input_args.split())
args

with open(args.configuration) as f:
    args = types.SimpleNamespace(**yaml.safe_load(f), **(vars(args)))
args.batch_size = args.cellpainting_args['batch_size']
args.compounds_per_batch = args.cellpainting_args['compounds_per_batch']

utils.set_global_seed(args.seed)

Namespace(seed=1234, runmode='snnl', configuration='hyperparameters/autoencoder_cellpainting-2.yaml')

In [7]:
trn_file_sz = args.cellpainting_args['train_end'] - args.cellpainting_args['train_start']
val_file_sz = args.cellpainting_args['val_end']   - args.cellpainting_args['val_start']
tst_file_sz = args.cellpainting_args['test_end']  - args.cellpainting_args['test_start']
smp_sz = args.cellpainting_args['sample_size']
buf_sz = args.cellpainting_args['compounds_per_batch']
bth_sz = args.batch_size
recs_per_batch = smp_sz * bth_sz * buf_sz

for file_sz in [tst_file_sz]:
    bth_per_epoch = file_sz // recs_per_batch
    print(f" - Each mini-batch contains {recs_per_batch/smp_sz} compounds with {smp_sz} samples per each compound : total {recs_per_batch} rows")
    print(f" - Number of {recs_per_batch} row full size batches per epoch: {bth_per_epoch}")
    print(f" - Rows covered by {bth_per_epoch} full size batches ({recs_per_batch} rows) per epoch:  {(file_sz // recs_per_batch) * recs_per_batch}")
    print(f" - Last partial batch contains : {file_sz % recs_per_batch} rows")
    print() 

 - Each mini-batch contains 100.0 compounds with 3 samples per each compound : total 300 rows
 - Number of 300 row full size batches per epoch: 12
 - Rows covered by 12 full size batches (300 rows) per epoch:  3600
 - Last partial batch contains : 0 rows



In [66]:
# args.project_name = 'CellPainting_Profiles'
# args.exp_id   = None
# args.exp_name = datetime.now().strftime('%m%d_%H%M')
# args.exp_desc = 'Autoencoder Training in SNNL mode'
# args.exp_date = datetime.now().strftime('%Y%m%d')
# wandb_run = init_wandb(args)
args.cellpainting_args['validation_path']
args.cellpainting_args['test_start'] = 0 
args.cellpainting_args['test_end'] = 346542
args.cellpainting_args['training_path'] = '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv'
args.cellpainting_args['validation_path'] = '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv'
args.cellpainting_args['train_start'] = 0
args.cellpainting_args['train_end'] = 312_000
args.cellpainting_args

'/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_test.csv'

{'sample_size': 3,
 'batch_size': 1,
 'compounds_per_batch': 100,
 'training_path': '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv',
 'validation_path': '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv',
 'train_start': 0,
 'train_end': 312000,
 'val_start': 0,
 'val_end': 24000,
 'test_start': 0,
 'test_end': 346542,
 'chunksize': None,
 'conversions': None,
 'iterator': True,
 'test_path': '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv'}

### Load Cell Painting Dataset

In [74]:
if args.dataset == 'cellpainting':
    print(f" load {args.dataset}")
    test_dataset = utils.CellpaintingDataset(type = 'test',    **args.cellpainting_args)
    test_loader = utils.InfiniteDataLoader(dataset=test_dataset, batch_size= args.batch_size,shuffle = False, num_workers = 0, collate_fn = utils.custom_collate_fn)

 load cellpainting
 Building CellPantingDataset from NOTEBOOK
    _init()_    -- filename:          /home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv
    _init()_    -- type :             test
    _init()_    -- start :            0
    _init()_    -- end :              346542
    _init()_    -- numrows :          346542


In [53]:
# input_shape = 784
# code_units =  20
args.temperature   = 0.00
args.loss_factor   = 1.0
args.snnl_factor   = 0.0e+00
args.learning_rate = 0.0e-00    ## 0.001
args.temperatureLR = 0.0e-00    ## 1e-4
# temperatureLR = 1e-03     ## 1e-4
print(f"   Units             {args.units}")
print(f"   Activations       {args.activations}")
print(f"   Latent dim        {args.code_units}")
print(f"   loss_factor       {args.loss_factor}")
print(f"   snnl_factor       {args.snnl_factor}")
print(f"   temperature       {args.temperature}")
print(f"   learning_rate     {args.learning_rate}")
print(f"   temperatureLR:    {args.temperatureLR}")

   Units             [['linear', 1471, 1024], ['relu', 0, 0], ['linear', 1024, 128], ['relu', 0, 0], ['linear', 128, 100], ['linear', 100, 128], ['relu', 0, 0], ['linear', 128, 1024], ['relu', 0, 0], ['linear', 1024, 1471]]
   Activations       ['relu', 'relu', 'sigmoid', 'relu', 'relu', 'relu']
   Latent dim        100
   loss_factor       1.0
   snnl_factor       0.0
   temperature       0.0
   learning_rate     0.0
   temperatureLR:    0.0


In [68]:
list_namespace(args)


command line parms : 
-----------------------
activations..............  ['relu', 'relu', 'sigmoid', 'relu', 'relu', 'relu']
batch_size...............  1

    cellpainting_args   (dict)
    ----------------------------
    batch_size...............  1
    chunksize................  None
    compounds_per_batch......  100
    conversions..............  None
    iterator.................  True
    sample_size..............  3
    test_end.................  346542
    test_path................  /home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv
    test_start...............  0
    train_end................  312000
    train_start..............  0
    training_path............  /home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv
    val_end..................  24000
    val_start................  0
    validation_path..........  /home/kevin/WSL-share

# Define Autoencoder Model

In [55]:
if args.runmode.lower() == "baseline":
    print(f"Defining model in baseline mode")
    model = Autoencoder(
        mode = "autoencoding",
        units=args.units,
        activations = args.activations,
        embedding_layer = 4,
        code_units  = args.code_units, 
        input_shape = args.input_shape, 
        sample_size = args.cellpainting_args['sample_size'],
        criterion   = torch.nn.MSELoss(reduction='mean'),
        loss_factor = args.loss_factor,
        learning_rate=args.learning_rate,
        adam_weight_decay = 0,
        use_scheduler = True,
        
        use_snnl = False,
        snnl_factor= 0.0,
        use_temp_scheduler = False
        )
elif args.runmode.lower() == "snnl":
    print(f"Defining model in SNNL mode ")
    model = Autoencoder(
        mode="latent_code",
        units=args.units,
        activations = args.activations,
        embedding_layer = 4,
        code_units = args.code_units,
        input_shape=args.input_shape,
        sample_size = args.cellpainting_args['sample_size'],
        criterion = torch.nn.MSELoss(reduction='mean'),
        loss_factor = args.loss_factor,
        learning_rate=args.learning_rate,
        adam_weight_decay = 0,
        use_scheduler = True,
        
        use_snnl=True,
        snnl_factor=args.snnl_factor,
        temperature=args.temperature,
        temperatureLR=args.temperatureLR,
        use_annealing= False,        
        use_sum = False,
        SGD_weight_decay= 0,
        use_temp_scheduler = False
        )
else:
    raise ValueError("Choose between [baseline] and [snnl] only.")

model.device

Defining model in SNNL mode 
    Model_init()_    -- AE - support for unsupervised in latent_code mode is True
    layer pair:    0  type:linear           input:   1471  output:   1024    weights: [1024, 1471]   
    layer pair:    1  type:relu             input:      0  output:      0    weights: [0, 0]   
    layer pair:    2  type:linear           input:   1024  output:    128    weights: [128, 1024]   
    layer pair:    3  type:relu             input:      0  output:      0    weights: [0, 0]   
    layer pair:    4  type:linear           input:    128  output:    100    weights: [100, 128]   
    layer pair:    5  type:linear           input:    100  output:    128    weights: [128, 100]   
    layer pair:    6  type:relu             input:      0  output:      0    weights: [0, 0]   
    layer pair:    7  type:linear           input:    128  output:   1024    weights: [1024, 128]   
    layer pair:    8  type:relu             input:      0  output:      0    weights: [0, 0]   
 

/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


device(type='cuda', index=0)

In [16]:
# display_model_summary(model)

# Load Model from saved checkpoint - Run data through model

In [75]:
epochs = 50
ex_runmode = 'snnl'
# ex_runmode = 'baseline'
ex_date = "20240516"
ex_title="singleOptimizer-2"
ex_title="snglOpt-100Ltnt"
# ex_title="firstTry"
ex_cpb = args.compounds_per_batch
ex_factor = 0
# ex_title="dropout04"
# ex_title="secondTry"
# title="autoencoder"
# title='Xentropy'
# model.name

In [76]:
# filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{epochs:03d}_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
# file_pattern = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_*_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{epochs:03d}_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
file_pattern = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_ep_*.pt"
print(file_pattern)

AE_snnl_20240516_snglOpt-100Ltnt_ep_*.pt


In [77]:
# filelist = glob.glob(os.path.join('ckpts',file_pattern))
filelist = glob.glob(file_pattern,root_dir = './ckpts')
filelist = sorted(filelist)
filelist
epochlist =sorted([int(x[-6:-3]) for x in filelist])
epochlist
# epochlist =[int(x[31:34]) for x in filelist]
# epochlist =[int(x[26:29]) for x in filelist]
# epochlist = [200]
# epochlist = [350]

['AE_snnl_20240516_snglOpt-100Ltnt_ep_025.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_050.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_075.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_100.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_125.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_150.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_175.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_200.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_225.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_250.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_275.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_300.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_325.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_350.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_375.pt',
 'AE_snnl_20240516_snglOpt-100Ltnt_ep_400.pt']

[25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400]

In [78]:
# = utils.load_model_from_ckpt(model, ex_mode, ex_timestamp, ex_title, epochs, ex_cpb, ex_factor )
output_file      = "/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_embedding_108_HashOrder.csv"
output_file_cols =  ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022','Metadata_Hash','Metadata_Bin', 'Metadata_Permiation']
output_file_cols += [f'Feature_{x:03d}' for x in range(100)]
len(output_file_cols)

108

In [79]:
write_header = True
for epochs in epochlist[-1:]:
# for filename in filelist:
    filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_ep_{epochs:03d}.pt"
    model = utils.load_model_from_ckpt(model, filename = filename, v1=True, verbose = False)
    if model is None:
        print(f"model is none")
        break
        
    mdl_outputs = utils.get_latent_representation(model, test_loader, embedding_layer = 4, verbose = False)
    df_output = pd.DataFrame.from_records(mdl_outputs, columns = output_file_cols)
    df_output.to_csv(output_file, mode = 'a', index = False, header = write_header)
    write_header = False    


 ==> Loaded from checkpoint AE_snnl_20240516_snglOpt-100Ltnt_ep_400.pt successfully. last epoch on checkpoint: 400

 embedding layer: 4 - model.layer: in: 128 out: 100
(300, 108) (300, 108)
(300, 108) (600, 108)
(300, 108) (900, 108)
(300, 108) (1200, 108)
(300, 108) (1500, 108)
(300, 108) (1800, 108)
(300, 108) (2100, 108)
(300, 108) (2400, 108)
(300, 108) (2700, 108)
(300, 108) (3000, 108)
(300, 108) (3300, 108)
(300, 108) (3600, 108)
(300, 108) (3900, 108)
(300, 108) (4200, 108)
(300, 108) (4500, 108)
(300, 108) (4800, 108)
(300, 108) (5100, 108)
(300, 108) (5400, 108)
(300, 108) (5700, 108)
(300, 108) (6000, 108)
(300, 108) (6300, 108)
(300, 108) (6600, 108)
(300, 108) (6900, 108)
(300, 108) (7200, 108)
(300, 108) (7500, 108)
(300, 108) (7800, 108)
(300, 108) (8100, 108)
(300, 108) (8400, 108)
(300, 108) (8700, 108)
(300, 108) (9000, 108)
(300, 108) (9300, 108)
(300, 108) (9600, 108)
(300, 108) (9900, 108)
(300, 108) (10200, 108)
(300, 108) (10500, 108)
(300, 108) (10800, 108)
(30

## Read Embedded Features CSV file

In [47]:
output_file = '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_embedding_108_HashOrder.csv'
# output_file = '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_embedding_108_HashOrder_training.csv'
# output_file = '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_embedding_108_HashOrder_test.csv'
# output_file = '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv'
output_file

'/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_embedding_108_HashOrder_training.csv'

In [48]:
df_input = pd.read_csv(output_file,usecols=['Metadata_Source'])

In [49]:
df_input.info()
df_input.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312000 entries, 0 to 311999
Data columns (total 1 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Metadata_Source  312000 non-null  object
dtypes: object(1)
memory usage: 2.4+ MB


(312000, 1)

In [50]:
df_input

,Metadata_Source
0,source_1
1,source_10
2,source_3
3,source_6
4,source_3
...,...
311995,source_6
311996,source_2
311997,source_10
311998,source_6


In [25]:
38143-34542 

3601

## Read embedding files

In [58]:
list_namespace(args)


command line parms : 
-----------------------
activations..............  ['relu', 'relu', 'sigmoid', 'relu', 'relu', 'relu']
batch_size...............  1

    cellpainting_args   (dict)
    ----------------------------
    batch_size...............  1
    chunksize................  None
    compounds_per_batch......  100
    conversions..............  None
    iterator.................  True
    sample_size..............  3
    test_end.................  33600
    test_start...............  30000
    train_end................  240000
    train_start..............  0
    training_path............  /home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv
    val_end..................  24000
    val_start................  0
    validation_path..........  /home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_test.csv

code_units...............  100
compounds_per_batch..

In [32]:
args.cellpainting_args['training_path']
training_file = '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv'
test_file =     '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_test.csv'
df_tmp = pd.read_csv(training_file, usecols = ['Metadata_Batch'])

'/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv'

In [33]:
df_tmp.shape

(312000, 1)

In [68]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 1482 entries, source_3 to 0.0157571067224056
dtypes: float64(1474), int64(3), object(5)
memory usage: 115.9+ KB


In [69]:
df_tmp

,source_3,CP_32_all_Phenix1,B40503aW,A15,JCP2022_095379,7406311667471038063,15,45.98,3.828206519335035,1.6625689669332602,...,-0.5097590814079832,0.3214959901211392,1.8371830042618016,0.8937381573947375,-1.4315948703930883,1.1371222646753114,-0.6841179949671937,-0.185220094884761,3.0298604878790028,0.0157571067224056
0,source_11,Batch2,EC000046,K04,JCP2022_009278,7406361908543180200,8,62.78,4.139637,1.797821,...,-1.040823,-0.648372,-0.625978,-0.083356,-0.340651,0.075124,-0.221786,-0.225378,-0.241621,-0.046198
1,source_11,Batch2,EC000063,K04,JCP2022_009278,7406361908543180200,8,62.78,4.139637,1.797821,...,0.357192,-0.154244,-0.570790,0.224279,0.611545,-0.066262,1.169602,-0.053950,-0.325021,0.084892
2,source_9,20211013-Run14,GR00003281,J15,JCP2022_009278,7406361908543180200,8,62.78,4.139637,1.797821,...,-0.502824,0.118865,0.362941,0.305544,0.190656,-0.185751,-0.163769,-0.032529,0.143611,0.068147
3,source_5,JUMPCPE-20210908-Run28_20210909_072022,AETJUM107,J10,JCP2022_045457,7406621140030190348,12,33.20,3.502550,1.521138,...,0.427760,-0.281399,-0.908818,-1.025111,0.391389,0.188497,0.106405,0.116903,-0.140219,-0.344818
4,source_11,Batch3,EC000139,G04,JCP2022_045457,7406621140030190348,12,33.20,3.502550,1.521138,...,-0.956507,-0.195724,1.868610,0.425072,-0.470012,-0.073765,-0.196747,-0.129325,0.377919,-0.044691
5,source_9,20210824-Run5,GR00003386,Z34,JCP2022_045457,7406621140030190348,12,33.20,3.502550,1.521138,...,0.659823,0.007205,-0.131672,-0.045983,1.053257,-0.288258,-0.145113,-0.226662,-0.308838,-0.470906
6,source_8,J2,A1170472,F17,JCP2022_066470,7406753887173400964,4,36.68,3.602232,1.564429,...,0.655410,0.062556,-0.210866,-0.039846,0.499055,-0.054781,0.003434,-0.117792,-0.025472,-0.263518
7,source_10,2021_06_08_U2OS_48_hr_run4,Dest210608-151729,L12,JCP2022_066470,7406753887173400964,4,36.68,3.602232,1.564429,...,-0.255979,-0.186363,0.741646,-0.126793,-0.258338,-0.291122,-0.581835,-0.277918,-0.297231,-0.377758
8,source_2,20210614_Batch_1,1053600889,L12,JCP2022_066470,7406753887173400964,4,36.68,3.602232,1.564429,...,0.175002,-1.044680,-0.654123,-1.013461,-0.408960,0.258432,-0.351150,-1.049283,-0.270976,-1.067156
9,source_9,20210914-Run9,GR00003302,O18,JCP2022_057572,7406772977290372296,8,67.15,4.206929,1.827046,...,-0.142636,-0.844037,0.450042,-1.153650,-0.391028,-0.267892,-0.352700,-0.229430,-0.148757,-0.474374


In [34]:
312000+34542

346542

## Write to csv and Excel files

In [63]:
timestamp = datetime.now().strftime('%m%d_%H%M%S')
csv_filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{metric}_metrics_{timestamp}.csv"
csv_filename = os.path.join('./results', csv_filename)
csv_filename

excel_filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{metric}_metrics_{timestamp}.xlsx"
excel_filename = os.path.join('./results', excel_filename)
excel_filename

'./results/AE_snnl_20240516_snglOpt-100Ltnt_cosine_metrics_0603_132740.csv'

'./results/AE_snnl_20240516_snglOpt-100Ltnt_cosine_metrics_0603_132740.xlsx'

In [64]:
excel_sheet = f"{'bl' if ex_runmode == 'baseline' else ex_runmode}_{timestamp}"
excel_sheet

'snnl_0603_132740'

In [65]:
df_metrics.to_csv(csv_filename, index = False)
df_metrics.to_excel(excel_filename, sheet_name=excel_sheet, index_label='id')

In [66]:
# mdl_outputs.embeddings[9].sum()
# mdl_outputs.y_pred.sum()
# mdl_outputs.compounds[:17]
df_metrics

,epochs,R2_score,mse_score,rmse_score,mae_score,inp_all_min,inp_all_max,inp_all_avg,inp_all_std,CL_inp_all_avg,...,out_same_avg,out_same_std,CL_out_same_avg,CL_out_same_std,out_diff_min,out_diff_max,out_diff_avg,out_diff_std,CL_out_diff_avg,CL_out_diff_std
25,25,0.523303,0.392870,0.609970,0.423861,0.854285,1.11344,0.980299,0.056047,0.980229,...,0.866111,0.204503,0.866111,0.421256,0.624749,1.195316,0.916052,0.125468,0.916052,0.362470
50,50,0.507878,0.424868,0.635628,0.438314,0.854285,1.11344,0.980299,0.056047,0.980229,...,0.886790,0.222100,0.886790,0.428929,0.593630,1.185306,0.950813,0.124884,0.950813,0.358343
75,75,0.560225,0.381529,0.600099,0.417053,0.854285,1.11344,0.980299,0.056047,0.980229,...,0.869482,0.177908,0.869482,0.375860,0.627031,1.164985,0.905690,0.120430,0.905690,0.345887
100,100,0.553688,0.364285,0.584641,0.405007,0.854285,1.11344,0.980299,0.056047,0.980229,...,0.914682,0.134494,0.914682,0.372484,0.699607,1.131309,0.932877,0.091578,0.932877,0.345280
125,125,0.488126,0.440674,0.649593,0.447693,0.854285,1.11344,0.980299,0.056047,0.980229,...,0.921641,0.220260,0.921641,0.444563,0.678530,1.205227,0.938426,0.107182,0.938426,0.348612
150,150,0.469629,0.460828,0.665301,0.459678,0.854285,1.11344,0.980299,0.056047,0.980229,...,0.821898,0.238417,0.821898,0.408789,0.566773,1.183600,0.874231,0.133824,0.874231,0.353339
175,175,0.461579,0.479671,0.679772,0.459909,0.854285,1.11344,0.980299,0.056047,0.980229,...,0.921744,0.169788,0.921744,0.359398,0.745519,1.280103,0.953966,0.111383,0.953966,0.368296
200,200,0.420078,0.520857,0.709850,0.469146,0.854285,1.11344,0.980299,0.056047,0.980229,...,0.851647,0.248691,0.851647,0.393026,0.739460,1.255333,0.956975,0.109070,0.956975,0.344850
225,225,0.337914,0.598103,0.763949,0.489046,0.854285,1.11344,0.980299,0.056047,0.980229,...,0.802677,0.217727,0.802677,0.353762,0.735547,1.290337,0.944080,0.119817,0.944080,0.337972
250,250,0.363615,0.573671,0.747721,0.478170,0.854285,1.11344,0.980299,0.056047,0.980229,...,0.860125,0.223971,0.860125,0.431239,0.625168,1.177510,0.943409,0.120508,0.943409,0.354945
